In [ ]:
!pip install ipython-autotime

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_data=pd.read_csv("drive/MyDrive/tweets/twitter_training.csv")
train_data=train_data.dropna()

In [ ]:
for col in train_data.columns:
  print(col)

2401
Borderlands
Positive
im getting on borderlands and i will murder you all ,


In [ ]:
train_data.rename(columns={2401:'id','Borderlands':'name','Positive':'sentiment','im getting on borderlands and i will murder you all ,':'text'},inplace=True)

In [ ]:
mapping = {
    'Positive': 1,
    'Negative': 0,
    'Neutral': 2,
    'Irrelevant': 4
}

In [ ]:
train_data['sentiment_label'] = [mapping[value] for value in train_data['sentiment']]

In [ ]:
train_data=train_data.drop(['2401','name'],axis=1)

In [ ]:
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download("punkt")
nltk.download("stopwords")
nltk.download('wordnet')
def preprocess_text(text):
    if not isinstance(text, str):  # Check if 'text' is not a string
        return ""  # Return an empty string for non-string values
    text=re.sub('<[^>]*>','',text)
    text=re.sub('[^a-zA-Z]',' ',text).lower()
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    preprocessed_text = ' '.join(words)
    return preprocessed_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
train_data['preprocessed_text']=train_data['text'].apply(preprocess_text)

In [ ]:
train_data.nunique()

sentiment                4
text                 69490
sentiment_label          4
preprocessed_text    59828
dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_data['preprocessed_text'],train_data['sentiment_label'],test_size=0.2,random_state=45)
#X_train,X_valid,y_train,y_valid=train_test_split(X_train,y_train,test_size=0.2,random_state=45)

In [ ]:
y_train.nunique()

4

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
vectorizer1=CountVectorizer()
vectorizer2=TfidfVectorizer()
X_train_bow=vectorizer1.fit_transform(X_train)
X_test_bow=vectorizer1.transform(X_test)
X_train_tfidf=vectorizer2.fit_transform(X_train)
X_test_tfidf=vectorizer2.transform(X_test)
clf1=SVC()
clf1.fit(X_train_bow,y_train)
y_predict_bow=clf1.predict(X_test_bow)
accuracy_bow=accuracy_score(y_test,y_predict_bow)
print("Prediction accuracy in BOW is : ",round(accuracy_bow,3)*100,'%')
clf2=SVC()
clf2.fit(X_train_tfidf,y_train)
y_predict_tfidf=clf2.predict(X_test_tfidf)
accuracy_tfidf=accuracy_score(y_test,y_predict_tfidf)
print("Prediction accuracy in tfidf is : ",round(accuracy_tfidf,3)*100,'%')


Prediction accuracy in BOW is :  86.5 %
Prediction accuracy in tfidf is :  92.2 %


In [ ]:
t="During Mohammed Azharuddin's time, even Zimbabwe seemed tough for India. Then MK Dhoni came and we started defeating Australia. And we because the world champions. When Manmohan Singh was the PM, even Pakistan seemed powerful. Today, after Modi's arrival we find China to be a lightweight. What's important is who is your captain."

t=preprocess_text(t)
t=[t]

In [ ]:
t2=vectorizer2.transform(t)

In [ ]:
pred1=clf2.predict(t2)
if pred1==0:
  print('Negative')
elif pred1==1:
  print('Positive')
elif pred1==2:
  print('Neutral')
else:
  print('Irrelevant')

Neutral
